# Ce projet fonctionne avec les versions des package suivants:
- pandas 2.2.2
- matplotlib 3.9.0
- seaborn 0.13.2
- folium 0.17.0
- geopandas 1.0.1
- shapely 2.0.5

# Démarche

Nous avons un jeu de données fournit par la ville de Paris.

## Connaitre notre jeu de données.
### On identifie:
1. les dimensions du dataset
2. le type de données
3. Les valeurs principales valeurs (Max, min, impossible, abérrantes)

### Approche:
Notre dataset étant un dataset des arbres de Paris, nous nous concentrons pour le nettoage de données (valeurs impossible et abérrantes) sur les dimensions des arbres(hauteur, circonférences).

#### On remarque:
- 200 137 lignes et 18 colonnes (18 variables potenielles à étudier)
- On remarque aussi des valeurs Max pour la hauteur de 881818m soit 881.818 kms et circonférence de 250255 cm soit 2502.55m. ce sont des valeurs impossible
- Idem pour les valeurs min. Un arbre ne peu pas exister et avoir une hauteur ou une circonférence de zero. Donc se sont des valeurs impossible.

### Valeur nulles
- On comptabilise ensuite le nombre de valeurs null pour chaque variables
- On réalise une représentation graphique pour visualiser quelles variables ont le plus de valeur nulles ( ici sous forme de heatmap)
    - On peut supprimer les colonnes:
    - 'id', 'numero', 'complement_addresse', 'id_emplacement',
    - 'libelle_francais','genre','espece','variete'
qui sont null quasiement à 100% donc inutile pour nous ou non pertinent pour notre analyse.

### Nettoyage des données

une fois les colonnes non utilisé supprimées, on traite les valeurs impossible.<br/>
<br/>
#### Valeurs impossibles<br/>
- La hauteur :<br/>
Detailer les pourquoi des valeurs imossible
**En 2021 un des arbres le plus haut à Paris faisait 30m**<br/>
https://agriculture.gouv.fr/lun-des-plus-grands-arbres-de-paris-veille-sur-le-78-rue-de-varenne<br/>
**on peut retrouver une data d'un arbre de 45m en 2006**<br/>
https://public.opendatasoft.com/explore/dataset/arbresremarquablesparis2011/table/?flg=fr-fr&sort=arbres_hauteur en m.<br/>
On prendra ne hauteur max de 50 m dans notre cas.<br/>
<br/>
- La circonférence :<br/>
**En 2006 la circonférence max des arbres à Paris était de 740 cm**<br/>
https://public.opendatasoft.com/explore/dataset/arbresremarquablesparis2011/table/?flg=fr-fr&sort=arbres_hauteurenm<br/>
https://www.unjourdeplusaparis.caom/paris-vert/arbres-remarquables-paris<br/>

Les arbres au dessus seront donc considéré comme des valeurs impossibles.

- il s'agit là d'une règle de validation Simples: On définit une valeur maximum et minimum à ne pas dépasser.


#### Valeurs abérrantes<br/>

On utilise la méthode des écarts interquatile (IQR)

Pour identifer nos outliers, on affiche un boxplot de la hauteur et la circonférence avant et après.

### Analyse univariées

Maintenant que notre dataset est nettoyé de toute les valeurs impossible ainsi que des outliers, on peut travailler sur une analyse univarié. 

On réalise une analise univarié naive c'est à dire afficher les variable sous forme de graphe une à une pour determiner ensuite notre cible.

### Analyse bivariées

On réalise ensuite une matrice de corrélation pour comprendre le lien entre nos variables
La première matrice .corr() permet de valider la correlaion entre les variables quantitative (numerique).

La matrice de correlation montre un lien entre hauteur et circonférence

1. Variables Quantitatives :

- circonference_cm et hauteur_m ont une forte corrélation.
- Les autres associations entre variables quantitatives et qualitatives montrent des valeurs modérées ou élevées, ce qui mérite une attention particulière.

2. Variables Qualitatives :

- stade_developpement, domanialite, et remarquable montrent des associations élevées entre elles (valeurs proches de 1).

3. Variables Géographiques :

- arrondissement montre des associations modérées avec stade_developpement et domanialite.

### Cible d'Étude

Pour améliorer les tournées des agents d'entretien, nous devons cibler les variables qui influencent significativement la tâche d'entretien.

cibles potentielles:
- Variables Géographiques : arrondissement
- Caractéristiques des Arbres : circonference_cm et hauteur_m
- Stade de Développement : stade_developpement
- Domanialité : domanialite

### Proposition d'Améliorations

1. Analyse des Variables Géographiques :

- Arrondissement : La répartition des arbres par arrondissement peut indiquer les zones nécessitant plus d'attention.

2. Analyse des Caractéristiques des Arbres :

- Circonférence et Hauteur : Les arbres de grande taille peuvent nécessiter plus de temps pour l'entretien.

3. Analyse des Stades de Développement et Domanialité :

- Comprendre la distribution des stades de développement et des domanialités par arrondissement peut aider à prioriser les tournées.

## Ojectifs:
- Réaliser une analyse de données afin d'optimiser les tournées des personne d'entretien

## Installation Package

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import folium
import geopandas as gpd
from shapely.geometry import Point

## Chargement du DataSet à partir du fichier CSV

In [ ]:
data= pd.read_csv("datas/p2-arbres-fr.csv", sep=";") #chargement du fichier CSV dans la variables data

## Analyse du jeu de données

### Identifier les types de données

**1-connaitre les dimensions de son dataset**

In [ ]:
# dimension du dataset
dimension = data.shape
print(f"Le nombre de ligne du dataFrame {dimension[0]}")
print(f"Le nombre de colonne du dataFrame {dimension[1]}")

**2- connnaitre la composition de son dataset**

Il peut être utile connaitre le type de variables pour chaque colonnes.

In [ ]:
data.info()

On remarque egalement les colonnes avec les valeurs non-null et donc,une première indication de la quantité de valeur null pour chaque colonnes.<br/>

**3- compte les nombre de valeurs null pour chaque variables**

Par exemple, la colonne numero a 100% de ces valeurs null on peut sans aucun impact la supprimer de notre analyse.<br/>
Nous avancerons de cette manière pas à pas dans notre analyse de données pour proposer un axe d'amélioration des tournées.

Pour une meilleurs perception du nombre de valeurs nuls, on peut les exprimer en pourcentage.

In [ ]:
null_counts = data.isnull()
null_percentage = (null_counts.sum()/len(data))*100
for column, percentage in null_percentage.items():
    print(f'{column}: {percentage:.2f}%')

#### Variables quantitative

**4- identifier les principales valeurs de son dataset**

Pour les variables quantitative (numérique), il est bon de connaitre les principales statistiques tel que:<br/>
- la moyenne
- la valeur max
- la valeur mini
- la mediane...

In [ ]:
# description des variables quantitative
data.describe()

Ainsi on peut facilement identifier simplement des valeurs impossible tel que:<br/>
- Une circonference (max) de 250255 cm soit 2502m
- Une hauteur (max) de 881818 m soit 881 kms.
- Ou encore des valeur (min) égales à 0<br/>
On travailleras sur ces données pour le nettoyage de valeurs impossibles ou abérantes

#### Variables qualitative

Pour les variables qualitative (non-numérique), cela nous permet de connaitre:<br/>
- le nombre de valeurs unique (unique)
- la valeur la plus fréquente de chaque colonne (top)
- ainsi que la fréquence de cette valeur (freq)

In [ ]:
# description des variables qualitative
data.describe(include="object")

Cela pourra nous être utile dans le choix de nos représentations graphiques.

**5- Première représentation graphique: Diagramme en barres des valeurs nulles**

Précédement on a identifier et exprimé en pourcentage la quantité de valeur null.<br/>
On peut donc les classer par ordre croissant et le afficher sous forme de diagramme en barres.<br/>
Cela nous donne une visualisation plus direct des valeurs null de notre dataset

In [ ]:
null_percentage=data.isnull().mean()*100
null_percentage

In [ ]:
null_percentage=data.isnull().mean()*100
null_percentage=null_percentage.sort_values(ascending=False)

null_percentage_df = pd.DataFrame(null_percentage, columns=['Pourcentage de valeurs nulles'])

plt.figure(figsize=(14,8))
bar_plot = sns.barplot(x=null_percentage_df['Pourcentage de valeurs nulles'], y=null_percentage_df.index)

for index, value in enumerate(null_percentage):
    bar_plot.text(value+1, index, f'{value:.2f}%', color='black', va='center')
plt.title('Pourcentage de valeur nulles par colonnes')
plt.ylabel('')

Après avoir visualisé nos données et identifier les colonnes ayant le plus de valeur nulles,<br/>
on peut choisir les colonnes inutiles par manque de données ainsi que celle que l'on ne souhaite pas utiliser.

## Nettoyage des données

In [ ]:
data.drop(['numero','complement_addresse','type_emplacement','id_emplacement','lieu','genre','espece','variete'], axis=1,inplace=True)

#### Les valeurs impossibles

##### Valeurs Zero

On vérifie les colonne ayant des valeur égales à zero.

In [ ]:
zero_counts = (data==0).sum()
zero_counts=(zero_counts/len(data))*100
print(f'Liste des colonnes et proportions de valeurs egale à zero en % sur un total de {len(data)} individus')
zero_counts=zero_counts.round(2)

zero_counts

In [ ]:
zero_percentage =zero_counts.drop('id') #On exclus id de notre affichage mais on le garde dans notre DataFrame.
zero_percentage=zero_percentage.sort_values(ascending=False)
zero_percentage_df=pd.DataFrame(zero_percentage, columns=['Pourcentage de valeurs égales à zéro'])

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(zero_percentage_df, cmap='viridis', annot=True, fmt='.2f')
plt.title('Heatmap des valeurs égales à zéro')
plt.ylabel('Colonnes')

<b>Les 3 colonnes contenant des valeurs égales à zero comme on peut le voir ci-dessus sont par ordre décroissant les colonnes:<b/><br/>
- remarquable
- hauteur_m
- circonference_cm

On sait que la colonne "remarquable" contient zero ou 1 si l'arbre et remarquable ou non.<br/>
Cela n'est donc pas pertinent de traiter cette colonne. La valeur zero ici n'est pas une valeur abérante et signifie que l'arbre n'est pas remarquable.<br/>
En revanche, **supprimer les individus ayant une circonférence ou une hauteur de 0 est pertinent**.<br/>
Un arbre n'existe pas si sa hauteur ou sa circonférence est égale à 0.

On traite aussi les hauteur max et circonférence max<br/>

Au début de l'analyse de notre jeu de données: "identifier les principales valeurs de son dataset"<br/>
on avait remarquer des valeur impossible:<br/>
- Une circonference (max) de 250255 cm soit 2502m
- Une hauteur (max) de 881818 m soit 881 kms.<br/>
<br/>
<br/>
Voici comment nous traiterons ces valeurs:<br/>
<br/>
- La hauteur :<br/>
**En 2021 un des arbres le plus haut à Paris faisait 30m**<br/>
https://agriculture.gouv.fr/lun-des-plus-grands-arbres-de-paris-veille-sur-le-78-rue-de-varenne<br/>
**on peut retrouver une data d'un arbre de 45m en 2006**<br/>
https://public.opendatasoft.com/explore/dataset/arbresremarquablesparis2011/table/?flg=fr-fr&sort=arbres_hauteur en m.<br/>
**_On retiendra une hauteur max de 50 m dans notre cas._**
<br/>
<br/>
- La circonférence :<br/>
**En 2006 la circonférence max des arbres à Paris était de 740 cm**<br/>
https://www.unjourdeplusaparis.caom/paris-vert/arbres-remarquables-paris<br/>
**_On retiendra ici une circonference max de 740 cm dans notre cas._**
<br/>
<br/>
Les arbres au dessus seront donc considéré comme des valeurs impossibles.

In [ ]:
max_height=50

num_tall_trees = (data['hauteur_m']>max_height).sum()
num_tall_trees_null = (data['hauteur_m']==0).sum()
print(f"Il y a {num_tall_trees} arbres ayant une hauteur supèrieur à {max_height} m soit {((num_tall_trees/len(data))*100).round(2)} % du total des arbres")
print(f"et {num_tall_trees_null} arbres ayant une hauteur null soit {((num_tall_trees_null/len(data))*100).round(2)} % du total des arbres")

On nettoie notre jeu de données des individus de hauteur = 0 et de hauteur > 50m.

In [ ]:
# on retire donc dans un premier temps la hauteur abérante en considérant un hauteur max de 50 m
data = data[data['hauteur_m'] <= max_height]
data = data[data['hauteur_m'] != 0]
data.shape

On effectue le même nettoyage mais cette fois sur la circonférence.

In [ ]:
max_circonference = 740
sorted_circonference = (data['circonference_cm']>max_circonference).sum()
numb_circonference_null = (data['circonference_cm']==0).sum()

print(f"Il y a {sorted_circonference} arbres ayant une circonference de plus de {max_circonference} cm soit {((sorted_circonference/len(data))*100).round(2)} % du total des arbres")
print(f"et {numb_circonference_null} arbres ayant une circonference null soit {((numb_circonference_null/len(data))*100).round(2)} % du total des arbres")

On nettoie notre jeu de données des individus de circonférence = 0 et de circonférence > 740cm.

In [ ]:
# on retire la circonférence supèrieur à max_circonférence.
data = data[data['circonference_cm'] <= max_circonference]
data = data[data['circonference_cm'] != 0]

In [ ]:
print(f'Notre jeu de donnée nettoyé contient {len(data)} arbres')

Les valeurs impossibles sont traitées, nous allons travailler sur les valeurs abérantes

#### Les valeurs abérantes

Avant de traiter les valeurs abérantes nous pouvons les visaliser en utilisant une boite à moustache.<br/>
Une boîte à moustaches est un graphique qui permet de visualiser la distribution d'un jeu de données en utilisant cinq valeurs statistiques principales :
- le minimum,
- le premier quartile (Q1),
- la médiane (Q2),
- le troisième quartile (Q3)
- et le maximum.

In [ ]:
fig, axs= plt.subplots(1,2, figsize=(12,8))

sns.boxplot(data['circonference_cm'], ax=axs[0])
axs[0].set_title('Boite à moustache des circonférence en cm')

sns.boxplot(data['hauteur_m'], ax=axs[1])
axs[1].set_title('Boite à moustache des hauteur en m')

La boîte : Représente les valeurs entre Q1 et Q3.<br/>
La ligne à l'intérieur de la boîte représente la médiane (Q2)<br/>
Les moustaches : S'étendent des quartiles (Q1 et Q3) jusqu'aux valeurs minimum et maximum qui ne sont pas considérées comme des valeurs aberrantes.

On remarque des points en dehors des moustaches. Cela représentent les outliers.<br/>
Ce sont nos valeurs abérantes.<br/>
Ce sont ces valeurs que nous allons traiter<br/>
Pour ce faire nous allons utiliser la méthodes IQR.<br/>
c'est une mesure de dispersion statistique qui décrit l'étendue des valeurs situées entre le premier quartile (Q1) et le troisième quartile (Q3) d'un jeu de données.

In [ ]:
def detect_outliers_(data, column):
    Q1=data[column].quantile(0.25)
    Q3=data[column].quantile(0.75)
    IQR = Q3 -Q1
    lower_limit = max(0,Q1-1.5 * IQR)
    upper_limit = Q3 + 1.5 *IQR
    outliers = (data[column]<lower_limit) | (data[column]>upper_limit)
    return outliers, lower_limit, upper_limit

In [ ]:
outliers_hauteur, lower_hauteur, upper_hauteur = detect_outliers_(data,'hauteur_m')
outliers_circonference, lower_circonference, upper_circonference = detect_outliers_(data,'circonference_cm')
print(f'Limites pour hauteur_m: inférieure = {lower_hauteur}, supérieure = {upper_hauteur}')
print(f'Limites pour circonference_cm: inférieure = {lower_circonference}, supérieure = {upper_circonference}')

On ne garde donc que nos arbres correspondant aux limites définis par cette méthode.

In [ ]:
data=data[(data['hauteur_m']>=lower_hauteur) & (data['hauteur_m']<=upper_hauteur) & (data['circonference_cm']>=lower_circonference) & (data['circonference_cm']<upper_circonference) ]
print(f'Notre dataset contient à présent {len(data)} arbres')

In [ ]:
fig, axs= plt.subplots(1,2, figsize=(12,8))

sns.boxplot(data['circonference_cm'], ax=axs[0])
axs[0].set_title('Boite à moustache des circonférence en cm')

sns.boxplot(data['hauteur_m'], ax=axs[1])
axs[1].set_title('Boite à moustache des hauteur en m')

On remarque que de nouveau points (Outliers ou valeur abérantes) apparaissent.<br/>
Cela est normal en retirant les premières valeur abérantes, le nouveau jeu de donnée peut calculer de nouvelles valeur abérantes.<br/>
Si cela est nécessaire on peut les retirer de manière itérative.

## Analyse univariee

Nos données sont nettoyées des:<br/>
- valeurs impossibles
- valeurs abérantes

**Nous allons maintenant regarder les variables une à une pour determiner notre cible.**

### Distribution des arbres remarquables

In [ ]:
remarquable_counts = data['remarquable'].value_counts()

# Étiquettes pour le graphique en camembert
labels = ['Non remarquable', 'Remarquable']

plt.figure(figsize=(8, 8))
plt.pie(remarquable_counts, labels = labels, autopct='%1.1f%%', startangle = 90, colors =['orange','blue'])
plt.title("Distribution des valeurs remarquables")
plt.legend(loc="best")

On observe que 99.9% de nos arbres dans notre jeu de données nettoyé ne sont pas remarquable.

### Distribution des arbres par arrondissements

In [ ]:
# Distribution des arbres par arrondissement
arrondissement_counts = data['arrondissement'].value_counts().sort_values()
arrondissement_order = arrondissement_counts.index


plt.figure(figsize=(8, 8))
plt.pie(arrondissement_counts, labels=arrondissement_order, labeldistance=1.15, wedgeprops = { 'linewidth' : 3, 'edgecolor' : 'white' }, autopct='%1.1f%%');
plt.title("Distribution des arbres par arrondissement")
arrondissement_percent = (arrondissement_counts / len(data)) * 100

arrondissements_above_5_percent = arrondissement_percent[arrondissement_percent >= 5]
nombre_arrondissements = arrondissements_above_5_percent.count()
print(f'{nombre_arrondissements} arrondisements possède plus de 5% des arbres de notre jeu de données.')
somme_pourcentages = arrondissements_above_5_percent.sum()
print(f'Ces arrondissements représentent environ {somme_pourcentages:.2f}% du total des arbres de notre jeu de données.')


## Distribution de la hauteur des arbres

In [ ]:
fig, ax = plt.subplots(figsize =(12,9))

hauteur_bins= int(data['hauteur_m'].max().round(0))
print(f'max std {hauteur_bins}')
bins=np.histogram_bin_edges(data['hauteur_m'], bins=hauteur_bins)
ax.hist(data['hauteur_m'], bins=bins, edgecolor="black",linewidth=.2)

# Ajouter des titres et des labels
ax.set_title("Distribution des hauteurs des arbres")
ax.set_xlabel("Hauteur (mètres)")
ax.set_ylabel("Nombre d'arbres")

On constate une distribution multimodal de la hauteur des arbres.<br/>
cela peut se justifier par l'espèces des arbres.<br/>
Chaque espèce ne grandit pas à la même vitesse.<br/>
La distribution des hauteurs des arbres dans nos données montre plusieurs groupes distincts de hauteurs fréquentes, ce qui peut indiquer différentes espèces ou âges d'arbres.

## Distribution de la circonférence des arbres

In [ ]:
bins = range(int(data['circonference_cm'].min()), int(data['circonference_cm'].max()) + 10, 10)

fig, ax = plt.subplots(figsize =(12,9))
circonference_bins= int(data['circonference_cm'].max().round(0)/10)
bins=np.histogram_bin_edges(data['circonference_cm'], bins=circonference_bins)
ax.hist(data['circonference_cm'], bins=bins, edgecolor="black",linewidth=.2)

# Ajouter des titres et des labels
ax.set_title("Distribution des circonférence des arbres")
ax.set_xlabel("circonférence (cm)")
ax.set_ylabel("Nombre d'arbres")

La distribution est asymétrique vers la droite.<br/>
Cela signifie que la plupart des valeurs sont concentrées vers le bas de l'échelle (à gauche) avec une longue traîne qui s'étend vers des valeurs plus élevées (à droite).<br/>
La queue de la distribution s'étend bien au-delà des valeurs plus élevées, jusqu'à environ 250 centimètres, indiquant la présence de quelques arbres avec des circonférences très grandes.<br/>
Un grand nombre d'observations se concentrent autour d'une valeur moyenne et une minorité des observations s'étendent vers des valeurs extrêmes.

## Distribution des libelles francais

### Histogramme

In [ ]:
libelle_francais_counts=data['libelle_francais'].value_counts()

In [ ]:
plt.figure(figsize=(26,10))
libelle_francais_counts.plot(kind='bar')
plt.title('Distribution des arbres par libelle français')
plt.xlabel('libelle Francais')
plt.ylabel('Nombre d\'arbres')

On retrouve une distribtion asymetrique droite ici aussi.<br/>
Un petit nombre d'espèces d'arbres, comme le platane, le marronnier, et le tilleul, dominent avec des quantités extrêmement élevées par rapport aux autres.

### PieChart

Une représentation en PieChart peut être utile

In [ ]:
threshold_target=0.015 #Ajuste le % pour inclure plus ou moins de libellé Français dans "Autres"

total = libelle_francais_counts.sum()
threshold = threshold_target * total

libelle_francais_counts_grouped = libelle_francais_counts[libelle_francais_counts >= threshold]

clean_dataset = libelle_francais_counts_grouped.reset_index()
clean_dataset.columns=['libelle_francais','count']

libelle_francais_counts_with_others = libelle_francais_counts_grouped.copy()
libelle_francais_counts_with_others[f'Autre libelle Francais < {round(threshold_target*100,2)}%'] = libelle_francais_counts[libelle_francais_counts <threshold].sum()
number_not_other_libelle_francais=libelle_francais_counts_grouped.count()

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 9))

libelle_francais_counts_with_others.plot(kind='pie', autopct='%1.1f%%', startangle=90, ax=axes[0])
axes[0].set_title('Distribution des arbres par libellé Francais (avec "autres")')
axes[0].set_ylabel('')

clean_dataset.set_index('libelle_francais')['count'].plot(kind='pie', autopct='%1.1f%%', startangle=90, ax=axes[1])
axes[1].set_title('Distribution des arbres par libellé Francais (sans "autres")')
axes[1].set_ylabel('')


plt.tight_layout()


En retirant les arbres représentant moins de 1.5% de notre de notre jeu de données, on se rend compte que plus d'1/4 sont des Platane.

In [ ]:
percentage_not_other = 100-((libelle_francais_counts[libelle_francais_counts < threshold].sum()/total)*100)
print(f'{number_not_other_libelle_francais} libellés Français représentent environ {percentage_not_other.round(2)}% des valeurs total des données')

On va ce concentrer sur ces libellés qui représente la majorité de nos données.

On filtre donc en ne gardant que les libellés qui ont un pourcentage >= à notre valeur cible stockée dans "threshold".

In [ ]:
libelles_principaux = libelle_francais_counts[libelle_francais_counts >= threshold].index
data = data[data['libelle_francais'].isin(libelles_principaux)]

## Distribution stade de développement

### piechart

Il sera utiles de connaitre la réaprtition des stade de développement pour l'optimisation des tournées

In [ ]:
stade_developpement_counts=data['stade_developpement'].value_counts()

In [ ]:
labels =['Jeunes', 'Jeunes Adultes', 'Adultes','Mature']
label_mapping = {
    'J': 'Jeunes',
    'JA': 'Jeunes Adultes',
    'A': 'Adultes',
    'M': 'Mature'
}

pie_labels = [label_mapping[stage] for stage in stade_developpement_counts.index]
plt.figure(figsize=(12,9))
stade_developpement_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90, labels=pie_labels, wedgeprops = { 'linewidth' : 3, 'edgecolor' : 'white' })
plt.title('Distribution des arbres par stade de developpement')
plt.ylabel('')
plt.legend()

Une distribution sur la domanialité peut-être pertinent pour les tournées d'entretien.

In [ ]:
domanialite_counts=data['domanialite'].value_counts()
plt.figure(figsize=(12, 8))
domanialite_counts.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Distribution des arbres par domanialité')
plt.xlabel('Domanialité')
plt.ylabel('Nombre d\'arbres')
plt.xticks(rotation=45)
plt.show()

Om remarque une forte présence des arbres sur les alignements.

# Analyse bivariée

## Matrice de correlation

Cela permet d'identifier les relations linéaires entre les variables

In [ ]:
numeric_data = data.select_dtypes(include=[np.number])
correlation_matrix = numeric_data.corr()

In [ ]:
print(correlation_matrix)

Les point GPS ainsi que l'id ne sont pas pertinent pour une matrice de correlation. On ne les affichera pas.

### Interprétation

In [ ]:
numeric_data = data.select_dtypes(include=[np.number]).drop(columns=['id','geo_point_2d_a','geo_point_2d_b'])
correlation_matrix = numeric_data.corr()

In [ ]:
plt.figure(figsize=(14,8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title("Matrice de Correlation")
plt.show()

La principale observation est la forte corrélation positive entre la circonférence et la hauteur des arbres. 0.8<br/>
Les arbres plus hauts tendent à avoir une plus grande circonférence.<br/>
Les autres relations sont faibles et ne montrent pas de dépendances significatives entre les variables.<br/>
Cette matrice de corrélation peut être utilisée pour identifier les variables à explorer davantage dans des analyses futures.<br/>
<br/>
Les corrélations proches de 0 signifie qu'il n'y a pas de relation linéaire significative entre ces paires de variables.

In [ ]:
data.info()

### Relation entre la circonférence et la hauteur des arbres

In [ ]:
plt.figure(figsize=(14,8))
sns.scatterplot(x='circonference_cm', y='hauteur_m',data=data)
plt.title('Relation entre la circonférence et la hauteur des arbres')
plt.xlabel('Circonférence (cm)')
plt.ylabel('Hauteur (m)')

On remarque une correlation positive:
- Il y a une tendance générale à la hausse : les arbres avec une plus grande circonférence tendent à être plus hauts.
- La corrélation n'est pas parfaite, mais il y a une relation évidente entre ces deux variables.<br/>
<br/>
On observe aussi des clusters de points:
- Plusieurs lignes horizontales de points indiquent des hauteurs d'arbres communes, probablement dues à des espèces spécifiques ou à des pratiques de mesure.
- Les points se concentrent principalement autour de certaines valeurs de circonférence et de hauteur, indiquant des tailles d'arbres courantes.<br/>
<br/>
Outliers:
- Quelques points se trouvent en dehors de la tendance générale, mais ils ne sont pas nombreux.<br/>

**Le graphique confirme une relation positive entre la circonférence et la hauteur des arbres. Les lignes horizontales suggèrent des tailles standardisées ou des catégories d'espèces spécifiques**

# ANALYSE A LA CIBLE

Notre cible sera les stade de developpement des arbres.

Cette cible s'inscrit dans le "Plan Arbre:Les actions de Paris pour l’arbre et la nature en ville - Fiches-actions 2021-2026"
https://cdn.paris.fr/paris/2021/12/13/daf6cce214190a66c7919b34989cf1ed.pdf

ains que dans la "charte de l'arbre"
lien: https://cdn.paris.fr/paris/2021/12/13/9b5f4f79c5bf85d6441d5ad4be3a1669.pdf

### Distribution des arbres par arrondissement et stade de developpement

In [ ]:
stacked_data=data.groupby(['arrondissement','stade_developpement']).size().unstack(fill_value=0)
total_trees_per_arrondissement = stacked_data.sum(axis=1)
sorted_arrondissements=total_trees_per_arrondissement.sort_values().index
stacked_data = stacked_data.loc[sorted_arrondissements]
stacked_data.plot(kind='bar',stacked=True, figsize=(14,8), colormap='viridis')
plt.title('Distribution de la hauteur des arbres par arrondissement et stade de developpement')
plt.ylabel('Count')

Cela nous informe sur la repartition des arbres par arrondissement et par stade de developpement

In [ ]:
data.info()

In [ ]:
def distribution_trees_by_compareBy (data, target,compareBy, target_bins=1, colormap='tab20',legend_labels=None):
   
    if data[target].dtype == 'category' or data[target].dtype==object or data[target].dtype =='float64':
        stacked_data=data.groupby([target, compareBy],observed=False).size().unstack(fill_value=0)
    elif data[target].dtype in['int64']:
        bins=list(range(0,int(data[target].max())+target_bins,target_bins))
        data_add_columns_bins=f'{target}_bin'
        data[data_add_columns_bins]=pd.cut(data[target],bins=bins)
        stacked_data = data.groupby([data_add_columns_bins, compareBy], observed=False).size().unstack(fill_value=0)
    else:
        raise ValueError(f"Type de variable non supporté pour {target}")
        # Ajout des catégories manquantes
        
    stacked_data_percentage = stacked_data.div(stacked_data.sum(axis=1),axis=0)*100
    
    ax=stacked_data_percentage.plot(kind='bar', stacked=True, figsize=(12,8),colormap='tab20')

    if legend_labels:
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles, [legend_labels[label] for label in labels], title=compareBy, bbox_to_anchor=(1.05, 1), loc='upper left')
    else:
        plt.legend(title=compareBy, bbox_to_anchor=(1.05, 1), loc='upper left')
        
    plt.title(f'Distribution {target} des arbres par {compareBy}')
    plt.ylabel(f'Distribution par {compareBy}')
    plt.xlabel(target)
    
    #plt.legend(title=compareBy, bbox_to_anchor=(1.05,1),loc='upper left')
    plt.tight_layout()

In [ ]:
compareBy="stade_developpement"
label_mapping = {
    'J': 'Jeunes',
    'JA': 'Jeunes Adultes',
    'A': 'Adultes',
    'M': 'Mature'
}

In [ ]:
data_remarquable= data.copy()
data_remarquable['remarquable']= data_remarquable['remarquable'].replace({0.0:'non remarquable', 1.0:'remarquable'})

In [ ]:
distribution_trees_by_compareBy(data=data_remarquable,target='remarquable', compareBy=compareBy, colormap='tab20',legend_labels=None)

In [ ]:
distribution_trees_by_compareBy(data=data,target='domanialite', compareBy=compareBy, colormap='tab20')

In [ ]:
distribution_trees_by_compareBy(data=data,target='arrondissement', target_bins=1,compareBy=compareBy, colormap='tab20')

In [ ]:
distribution_trees_by_compareBy(data=data,target='hauteur_m', target_bins=5,compareBy=compareBy, colormap='tab20')

In [ ]:
distribution_trees_by_compareBy(data=data,target='circonference_cm', target_bins=10,compareBy=compareBy, colormap='tab20')

In [ ]:
distribution_trees_by_compareBy(data=data,target='libelle_francais', target_bins=1,compareBy=compareBy, colormap='tab20')

### Proportion d'arbres remarquables par domanialité

In [ ]:
plt.figure(figsize=(14,8))
remaquable_counts = data.groupby('domanialite')['remarquable'].sum().sort_values(ascending=False)
remaquable_counts.plot(kind='bar', color='skyblue')
plt.title('Proportion d\'arbres remarquables par domanialité')
plt.xlabel('Domanialité')
plt.ylabel('Nombre d\'arbres remarquables')

Le graphique met en évidence que les jardins sont les principaux contributeurs aux arbres remarquables.<br/>
Les autres domanialités, bien qu'importantes, ont une proportion beaucoup plus faible d'arbres remarquables.<br/>

### Carte de densité des arbres par coordonnées géographiques

In [ ]:
data['geometry'] = data.apply(lambda row: Point(row['geo_point_2d_a'], row['geo_point_2d_b']), axis=1)

In [ ]:
def create_map_with_markers(data,location =[48.858379601173475, 2.3460534179549994], zoom_start=12, color='green', add_popup=False, map_first_params=None,item_first_param=None, map_second_params=None,second_param=None):
    gdf = gpd.GeoDataFrame(data, geometry='geometry')
    gdf['point_size'] = gdf['hauteur_m']

    stade_mapping = {
        'J':'Jeune',
        'JA':'jeune Adulte',
        'A':'Adulte',
        'M':'Mature'
    }
    
    m = folium.Map(location =location, zoom_start=zoom_start)
    title = 'Carte de Tous les Arbres'
    if item_first_param:
        stade_full_name = stade_mapping.get(item_first_param, item_first_param)
        title = f'Carte des Arbres par {map_first_params}: {stade_full_name}'
    if second_param:
        title += f' et {map_second_params}: {second_param}'

    title_html = f'''
         <h3 align="center" style="font-size:20px"><b>{title}</b></h3>
         '''
    m.get_root().html.add_child(folium.Element(title_html))

    for idx, row in gdf.iterrows():
        popup = None
        if add_popup:
            popup_text = f"""
            <b>Libellé Français:</b> {row['libelle_francais']}<br>
            <b>Hauteur:</b> {row['hauteur_m']} m<br>
            <b>Circonférence:</b> {row['circonference_cm']} cm<br>
            <b>Remarquable:</b> {row['remarquable']}<br>
            <b>Latitude:</b> {row['geo_point_2d_b']}<br>
            <b>Longitude:</b> {row['geo_point_2d_a']}
            """
            popup = folium.Popup(popup_text, max_width=300)

        folium.CircleMarker(
            location =[row['geo_point_2d_a'],row['geo_point_2d_b']],
            radius=row['point_size']/2,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.5,
            popup=popup
    ).add_to(m)
    legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 250px; height: 75px; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white;
                 ">
     &nbsp; Légende <br>
     &nbsp; Taille du point: Hauteur de l'arbre <br>
     &nbsp; <i class="fa fa-circle" style="color:{}"></i> Arbres
     </div>
     '''.format(color)
    
    m.get_root().html.add_child(folium.Element(legend_html))
    return m

In [ ]:
data.info()

In [ ]:
m=create_map_with_markers(data, color='blue')
m

In [ ]:
map_first_params = 'stade_developpement'
map_second_params = 'domanialite'
data[map_first_params]=data[map_first_params].fillna('manquant')
data[map_second_params]=data[map_second_params].fillna('manquant')
if map_first_params == 'remarquable':
    data['remarquable']= data['remarquable'].replace({0.0:'non remarquable', 1.0:'remarquable'})

In [ ]:
if isinstance(data, pd.DataFrame):
    
    if isinstance(map_first_params, str) and map_first_params in data.columns:
        
        items_list=data[map_first_params].unique()
        
        for item in items_list:
            if pd.isna(item):
                m=create_map_with_markers(data,add_popup=True, item_first_param='NaN', map_first_params=map_first_params)
                display(m)
            else:
                data_map = data[data[map_first_params] == item]
                m=create_map_with_markers(data_map,add_popup=True, item_first_param=item,map_first_params=map_first_params)
                display(m)
    else:
        print(f'{map_first_params} n\'est pas une colonne valide dans le DataFrame.')
else:
    print("L\'objet 'data' n\'est pas un DataFrame")

In [ ]:
item_first_param=input(f'Merci de choisir le premier paramètre de votre graphique provenant de "{map_first_params}" dans la liste suivante: {data[map_first_params].unique()}')
for item in data[map_first_params].unique():
    if isinstance(item,str) and item.lower()==item_first_param:
        item_first_param=item
        break
print(f' Vous avez saisie: {item_first_param}')
data_filtered = data[data[map_first_params]==item_first_param]
item_second_param=input(f'Choisir le second paramètre de votre graphique provenant de "{map_second_params}" dans la liste suivante filtré par "{item_first_param}": {data_filtered[map_second_params].unique()}')
for item in data[map_second_params].unique():
    if isinstance(item,str) and item.lower()==item_second_param:
        item_second_param=item
        break

print(f' Vous avez saisie: {item_second_param}')
data_map= data[(data[map_first_params]==item_first_param) & (data[map_second_params]==item_second_param)]
data_map
print(f'nombre d\'individu\n{len(data_map)}')
m=create_map_with_markers(data_map,add_popup=True, item_first_param=item_first_param, map_first_params=map_first_params, map_second_params=map_second_params,second_param=item_second_param)
m

In [ ]:
data.shape

Avec cette carte nous pouvons identifier rapidement les arbres en fonction des paramètres map_first_params et map_second_params que l'on peut choisir comme on le souhaite.<br/>
Cela va nous permettre de visualiser au mieux ou sont situéles arbres pour les tournées d'entretien afin d'atteindre les objectifs du "Plan Arbre" et de la charte.

On notera que notre jeu de données analysé a été nettoyer des données impossible et abérantes. Pour une analyse collant au plus près du terrain, il conviendra de compléter ces données (Valeur égales à zero, NaN ou supérieurs aux hauteur et circonférence définis pour les outliers)

# Optimisation des tournées: Algorithmes de Path

Un algorithme d'optimisation de path en analyse de données est une technique utilisée pour trouver le chemin le plus optimal ou le plus efficace entre différents points.

## Dans le cas de l'optimisation de tourner pour l'entretien des arbres:

Pour l'optimisation des tournées d'entretien des arbres, des algorithmes d'optimisation de chemin peuvent être utilisés pour minimiser le temps et les coûts associés aux déplacements entre différents arbres ou sites à entretenir. Voici quelques méthodes et algorithmes spécifiques qui peuvent être appliqués dans ce contexte :

### Problème du Voyageur de Commerce (TSP)

Le problème du voyageur de commerce est un problème d'optimisation classique où l'objectif est de trouver le chemin le plus court passant par un ensemble de points (dans ce cas, les arbres) et revenant au point de départ. Plusieurs algorithmes peuvent être utilisés pour résoudre le TSP


### Méthode de Clarke-Wright (Savings Algorithm)

Cette méthode est utilisée pour optimiser les routes dans un problème de tournées de véhicules (VRP), qui est une généralisation du TSP. Elle commence par une solution initiale où chaque arbre est visité indépendamment et fusionne ensuite les routes de manière itérative pour réduire le coût total.

### Algorithmes Heuristiques et Métaheuristiques

Ces algorithmes sont souvent utilisés pour trouver des solutions quasi-optimales à des problèmes complexes comme l'optimisation des tournées d'entretien

# Conclusion

Ces différentes approches permettrons d'organiser de manière efficace vos tournées d'entretien.

## Pour aller plus loin:

En fonction de la répartition des équipes sur le territoire et de leur point de départ, je peux optimiser les tournées par arrondissement si cela est plus efficace pour vous.